In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder, normalize
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier

# !pip install tensorflow==2.1.0
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, Conv1D, MaxPool1D
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tensorflow.keras.losses import KLDivergence, mean_squared_error

import keras
from keras.utils import np_utils

In [2]:
feature_dim = 32
encoding_dim = 16
ae_epoch = 1
clf_epoch = 1
batch_size = 32

In [3]:
df = pd.read_csv('kdd_train.csv')
df.head(5)

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,labels
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal
1,0,udp,other,SF,146,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
2,0,tcp,private,S0,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune
3,0,tcp,http,SF,232,8153,0,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
4,0,tcp,http,SF,199,420,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal


In [4]:
le = LabelEncoder()

df['labels'] = le.fit_transform(df['labels'])
df['protocol_type'] = le.fit_transform(df['protocol_type'])
df['service'] = le.fit_transform(df['service'])
df['flag'] = le.fit_transform(df['flag'])

In [5]:
x = df.drop('labels', axis=1)
y = df.loc[:, ['labels']]

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=.2, random_state=42)

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_train = pd.DataFrame(X_train)
X_test = sc.fit_transform(X_test)
X_test = pd.DataFrame(X_test)

In [6]:
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

In [7]:
X_train.shape

(100778, 41)

In [8]:
#KMEANS = 
DTC = DecisionTreeClassifier()
rfe = RFE(DTC, n_features_to_select= feature_dim).fit(X_train,y_train)
indices = np.where(rfe.support_==True)[0]
features = X_train.columns.values[indices]
X_train, X_test = X_train[features], X_test[features]

In [9]:
class HybridHuberLoss(tf.keras.losses.Loss):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
    def call(self, y_true, y_pred):
        mse = mean_squared_error(y_true, y_pred)
        kld = KLDivergence()
        kld_loss = kld(y_true, y_pred)
        return tf.where(mse<kld_loss, kld_loss, mse)
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}

In [10]:
def LRschedulerCLF(epoch, lr=0.01):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

def LRschedulerAE(epoch):
    initial_lrate = 0.01
    drop = 0.005
    epochs_drop = 5.0
    lrate = initial_lrate * math.pow(drop,  
           math.floor((1+epoch)/epochs_drop))
    return lrate
    
clf_lr = tf.keras.callbacks.LearningRateScheduler(LRschedulerCLF)
ae_lr = tf.keras.callbacks.LearningRateScheduler(LRschedulerAE)

In [11]:
input_dim = X_train.shape[1]
ae_input_layer = Input(shape=(input_dim, ))

enc = Dense(32, activation="swish", activity_regularizer=regularizers.l1(10e-5))(ae_input_layer)
enc = BatchNormalization()(enc)
enc = Dense(encoding_dim, activation="swish")(enc)

dec = BatchNormalization()(enc)
dec = Dense(32, activation="swish")(dec)
dec = BatchNormalization()(enc)
dec = Dense(input_dim, activation="swish")(dec)

autoencoder = Model(inputs=ae_input_layer, outputs=dec)
encoder = Model(inputs=ae_input_layer, outputs=enc)

In [12]:
autoencoder.compile(optimizer='adam', loss="mean_squared_error", metrics=['accuracy'])

In [13]:
history = autoencoder.fit(X_train, X_train, 
                    epochs=ae_epoch,
                    batch_size=batch_size,
                    shuffle=True,
                    validation_data=(X_test, X_test),
                    callbacks = [ae_lr],
                    verbose=1).history

3150/3150 [==============================] - 5s 1ms/step - loss: 0.3105 - accuracy: 0.5776 - val_loss: 0.3617 - val_accuracy: 0.5306


In [14]:
encoder.trainable = False

input_layer = Input(shape=(feature_dim, ))

encoding = encoder(input_layer, training=False)

layer1 = Dense(64, activation="relu")(encoding)
layer1 = BatchNormalization()(layer1)
layer1 = Dropout(0.2)(layer1)

layer2 = Dense(64, activation="relu")(layer1)
layer2 = BatchNormalization()(layer2)
layer2 = Dropout(0.3)(layer2)

layer3 = Dense(128, activation="relu")(layer2)
layer3 = BatchNormalization()(layer3)
layer3 = Dropout(0.3)(layer3)

layer4 = Dense(128, activation="relu")(layer3)
layer4 = BatchNormalization()(layer4)
layer4 = Dropout(0.2)(layer4)

output_layer = Dense(23, activation="softmax")(layer4)

xnn = Model(inputs=input_layer ,outputs=output_layer)

In [15]:
xnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', 'Precision', 'AUC'])
xnn.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 32)]              0         
_________________________________________________________________
model_1 (Functional)         (None, 16)                1712      
_________________________________________________________________
dense_4 (Dense)              (None, 64)                1088      
_________________________________________________________________
batch_normalization_3 (Batch (None, 64)                256       
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 64)                4160      
_________________________________________________________________
batch_normalization_4 (Batch (None, 64)                256 

In [16]:
history = xnn.fit(X_train, y_train,
                    epochs=clf_epoch,
                    batch_size=batch_size,
                    shuffle=True,
                    validation_data=(X_test, y_test),
                    callbacks=[ae_lr],
                    verbose=1).history

3150/3150 [==============================] - 12s 3ms/step - loss: 0.2122 - accuracy: 0.9412 - precision: 0.9508 - auc: 0.9961 - val_loss: 0.1912 - val_accuracy: 0.9436 - val_precision: 0.9513 - val_auc: 0.9961


In [17]:
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score

In [18]:
y_preds = xnn.predict(X_test)
y_preds = np.round_(y_preds)

In [19]:
print(precision_score(y_test, y_preds, average='micro', zero_division=0))
print(precision_score(y_test, y_preds, average='weighted', zero_division=0))

0.951345922057051
0.9508500995653361


In [20]:
print(f1_score(y_test, y_preds, average='micro', zero_division=0))
print(f1_score(y_test, y_preds, average='weighted', zero_division=0))

0.9455525606469003
0.9416393947798697


In [21]:
import numpy as np


def plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True):
  
    import matplotlib.pyplot as plt
    import numpy as np
    import itertools

    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.show()
    plot_confusion_matrix(cm           = np.array([[ 1098,  1934,   807],
                                              [  604,  4392,  6233],
                                              [  162,  2362, 31760]]), 
                      normalize    = True,
                      target_names = ['high', 'medium', 'low'],
                      title        = "Confusion Matrix, Normalized")

In [ ]:
plot_confusion_matrix(cm           = np.array([[ 1098,  1934,   807],
                                              [  604,  4392,  6233],
                                              [  162,  2362, 31760]]), 
                      normalize    = False,
                      target_names = ['high', 'medium', 'low'],
                      title        = "Confusion Matrix")